In [1]:
import requests
import pandas as pd
import os
import shutil
from bs4 import BeautifulSoup
import zipfile
import shutil
import glob
from datetime import datetime

In [2]:
def download_file(url, filename):
    if not os.path.isfile(filename):
        print('Downloading File')
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                for chunk in response:
                    file.write(chunk)
    else:
            print('file exists')

In [3]:
def get_nvd_json_files(download_path):
    link = "https://nvd.nist.gov/vuln/data-feeds#JSON_FEED"
    re = requests.get(link)
    soup = BeautifulSoup(re.text)
    i=0
    for row in soup.find('table').find('tbody').find_all('tr'):

        if i%3==0:
            name=row.find_all('td')[0].text
            date=(row.find_all('td')[1].text).split('-')[0].strip().replace(':','_').replace(' ','--').replace('/','_')
            file=row.find_all('td')[2].a.get('href').replace('meta','json.zip')
            name=name+'--'+date
            download_file(url=file, filename=os.getcwd()+os.sep+'NVD'+os.sep+name+'.zip')
            with zipfile.ZipFile(os.getcwd()+os.sep+'NVD'+os.sep+name+'.zip') as zf:
                zf.extractall(os.getcwd()+os.sep+'NVD'+os.sep)
            os.remove(os.getcwd()+os.sep+'NVD'+os.sep+name+'.zip')
        i=i+10    

In [6]:
download_path=r'D:\Threat_intel_services\rhym_04_06_2020\TIP_automation\ISR\NVD'
get_nvd_json_files(download_path)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Threat_intel_services\\rhym_04_06_2020\\TIP_automation\\Data_crawling\\NVD\\NVD\\CVE-Modified--6_8_2020--8_03_19--AM.zip'

In [341]:
def fetch_configuration(file):
    df=pd.read_json(file)
    cve_items=df.CVE_Items.apply(pd.Series)
    cve=cve_items.cve.apply(pd.Series)
    metadata=cve.CVE_data_meta.apply(pd.Series)
    config=cve_items.configurations.apply(pd.Series)
    
    config['cpe_uri']=None
    config['CVE_ID']=None
    for index,row in config.iterrows():
        config['CVE_ID'][index]=metadata['ID'][index]
        if len(row['nodes'])>0:
            try:
                config['cpe_uri'][index]=row['nodes'][0]['cpe_match']
            except KeyError:
                try:
                    config['cpe_uri'][index]=row['nodes'][0]['children'][0]['cpe_match']
                except KeyError:
                      pass
    initial=True
    for index,row in config.iterrows():
        print(index)
        if row['cpe_uri'] is not None:
            for i in row['cpe_uri']:
                df=pd.DataFrame([i['cpe23Uri'],row['CVE_ID']]).T
                if initial:
                    result=df
                    initial=False
                else:
                    result=result.append(df)           
       
    
    temp=result[0].str.split(':',n=10,expand=True)
    col=['CPE','version','part','vendor','product','update','Language','Software_edition','Target_software','Target_hardware','Other']
    temp.columns=col
    temp['CVE_ID']=result[1]
    temp['Date']=datetime.now()
    return temp


In [330]:
for file in glob.glob(r'D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\*.json'): 
    #reading a json file
    df=pd.read_json(file)
    cve_items=df.CVE_Items.apply(pd.Series)
    cve=cve_items.cve.apply(pd.Series)
    # fetching metadata
    metadata=cve.CVE_data_meta.apply(pd.Series)
    #getting problem
    problem=cve.problemtype.apply(pd.Series)
    #fetching description
    des=problem.problemtype_data.apply(pd.Series)
    des=des[0].apply(pd.Series)
    des=des['description'].apply(pd.Series)
    des=(des[0].apply(pd.Series))
    des.drop(columns=0,axis=1,inplace=True)
    #fetching impact
    impact=cve_items.impact.apply(pd.Series)
    
    #fetching explanation about cve
    text=cve.description.apply(pd.Series)
    text=text.description_data.apply(pd.Series)
    text=text[0].apply(pd.Series)
    #references
    ref=cve.references.apply(pd.Series)
    ref=ref.reference_data
    main_url=[]
    for r in ref:
        url=[]
        for data in r:
            try:
                url.append(data['url'])
            except KeyError:
                url.append(None)
        main_url.append('\n'.join(url))
    reference_df=pd.DataFrame(main_url,columns=['references'])
    for i in impact.columns:
        print(i)
        if 'baseMetricV2' in i:
            df_v2=impact['baseMetricV2'].apply(pd.Series)
            df1=df_v2['cvssV2'].apply(pd.Series)  
            df1.drop(columns=[0,'vectorString'],axis=1,inplace=True)
            df1=df1.add_prefix('v2_')
            df1=pd.concat([df_v2,df1],axis=1)
        
        elif 'baseMetricV3' in i:
            df_v3=impact['baseMetricV3'].apply(pd.Series)
            df2=df_v3['cvssV3'].apply(pd.Series)
            df2.drop(columns=[0,'vectorString'],axis=1,inplace=True)
            df2=df2.add_prefix('v3_')
            df2=pd.concat([df_v3,df2],axis=1)
        else:
            continue
    df_final=pd.concat([df1,df2],axis=1)
    impact=df_final.drop([0,'acInsufInfo','cvssV2','cvssV3'],axis=1)
    df_NVD=pd.concat([cve[['data_type','data_format','data_version']],metadata,des,text['value'],cve_items[['publishedDate','lastModifiedDate']],impact,reference_df],axis=1)
    df_NVD['Date']=datetime.now()
    df_NVD.to_csv(r'D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV'+os.sep+file.split('\\')[-1].split('.json')[0]+'.csv',index=False)
    cpe=fetch_configuration(file)
    cpe.to_csv(r'D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV'+os.sep+file.split('\\')[-1].split('.json')[0]+'_CPE'+'.csv',index=False)
    
    print(file.split('\\')[-1]+'_completed')

baseMetricV2
baseMetricV3
nvdcve-1.1-2002.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2003.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2004.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2005.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2006.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2007.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2008.json_completed
baseMetricV2
baseMetricV3
nvdcve-1.1-2009.json_completed
baseMetricV2
baseMetricV3


KeyboardInterrupt: 

In [350]:
import glob
initial=True
for i in glob.glob(r'D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\*.csv')[:-2]:
    print(i)
    df=pd.read_csv(i)
    if initial:
        result=df
        initial=False
    else:
        result=result.append(df)

D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2002.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2003.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2004.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2005.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2006.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2007.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2008.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2009.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2010.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2011.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2012.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2013.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2014.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2015.csv
D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\

In [353]:
result.to_csv(r"D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve_consolidated.csv",index=False)

In [358]:
result[result['ID']=='CVE-2010-3457']

,data_type,data_format,data_version,ID,ASSIGNER,lang,value,value.1,publishedDate,lastModifiedDate,...,v3_baseScore,v3_baseSeverity,v3_confidentialityImpact,v3_integrityImpact,v3_privilegesRequired,v3_scope,v3_userInteraction,v3_version,references,Date
3343,CVE,MITRE,4.0,CVE-2010-3457,cve@mitre.org,en,CWE-79,Multiple cross-site scripting (XSS) vulnerabil...,2010-09-17T20:00Z,2020-05-28T15:50Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://packetstormsecurity.org/1009-exploits/s...,2020-06-08 15:13:11.550920


In [355]:
df=pd.read_csv(r"D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-modified.csv")

In [331]:
def fetch_configuration(file):
    df=pd.read_json(file)
    cve_items=df.CVE_Items.apply(pd.Series)
    cve=cve_items.cve.apply(pd.Series)
    metadata=cve.CVE_data_meta.apply(pd.Series)
    config=cve_items.configurations.apply(pd.Series)
    
    config['cpe_uri']=None
    config['CVE_ID']=None
    for index,row in config.iterrows():
        config['CVE_ID'][index]=metadata['ID'][index]
        if len(row['nodes'])>0:
            try:
                config['cpe_uri'][index]=row['nodes'][0]['cpe_match']
            except KeyError:
                try:
                    config['cpe_uri'][index]=row['nodes'][0]['children'][0]['cpe_match']
                except KeyError:
                      pass
    initial=True
    for index,row in config.iterrows():
        if row['cpe_uri'] is not None:
            for i in row['cpe_uri']:
                df=pd.DataFrame([i['cpe23Uri'],row['CVE_ID']]).T
                if initial:
                    result=df
                    initial=False
                else:
                    result=result.append(df)           
       
    
    temp=result[0].str.split(':',n=10,expand=True)
    col=['CPE','version','part','vendor','product','update','Language','Software_edition','Target_software','Target_hardware','Other']
    temp.columns=col
    temp['CVE_ID']=result[1]
    temp['Date']=datetime.now()
    return temp


In [336]:
cpe=fetch_configuration(file=r"D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\nvdcve-1.1-2011.json")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
cpe.to_csv(r"D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\NVD_CSV\nvdcve-1.1-2011_CPE.csv")

In [337]:
file=r"D:\MY_STUFF\rhym\TIP_automation\ISR\NVD\nvdcve-1.1-2011.json"

In [338]:
df=pd.read_json(file)
cve_items=df.CVE_Items.apply(pd.Series)
cve=cve_items.cve.apply(pd.Series)
metadata=cve.CVE_data_meta.apply(pd.Series)
config=cve_items.configurations.apply(pd.Series)

In [340]:
config['cpe_uri']=None
config['CVE_ID']=None
for index,row in config.iterrows():
    config['CVE_ID'][index]=metadata['ID'][index]
    if len(row['nodes'])>0:
        try:
            config['cpe_uri'][index]=row['nodes'][0]['cpe_match']
        except:
            config['cpe_uri'][index]=row['nodes'][0]['children'][0]['cpe_match']
initial=True
for index,row in config.iterrows():
    print(index)
    if row['cpe_uri'] is not None:
        for i in row['cpe_uri']:
            df=pd.DataFrame([i['cpe23Uri'],row['CVE_ID']]).T
            if initial:
                result=df
                initial=False
            else:
                result=result.append(df)           

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27


KeyboardInterrupt



In [267]:
temp=result[0].str.split(':',n=10,expand=True)
col=['part','vendor','product','version','update','edition','Language','Software_edition','Target_software','Target_hardware','Other']
temp.columns=col
temp['CVE_ID']=result[1]

In [268]:
temp

,part,vendor,product,version,update,edition,Language,Software_edition,Target_software,Target_hardware,Other,CVE_ID
0,cpe,2.3,o,freebsd,freebsd,4.2,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,freebsd,freebsd,4.3,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,freebsd,freebsd,4.4,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,freebsd,freebsd,4.5,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,freebsd,freebsd,4.6,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,freebsd,freebsd,4.7,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,linux,linux_kernel,2.4.1,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,linux,linux_kernel,2.4.2,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,linux,linux_kernel,2.4.3,*,*,*,*,*:*:*,CVE-2003-0001
0,cpe,2.3,o,linux,linux_kernel,2.4.4,*,*,*,*,*:*:*,CVE-2003-0001


In [200]:
initial=True
for index,row in config.iterrows():
    if row['cpe_uri'] is not None:
        for i in row['cpe_uri']:
            df=pd.DataFrame([i['cpe23Uri'],row['CVE_ID']]).T
            if initial:
                result=df
                initial=False
            else:
                result=result.append(df)           

In [218]:
len(col)

11

In [201]:
col=['part','vendor','product','version','update','edition','Language','Software_edition','Target_software','Target_hardware','Other']

,0,1
0,cpe:2.3:o:freebsd:freebsd:4.2:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:freebsd:freebsd:4.3:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:freebsd:freebsd:4.4:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:freebsd:freebsd:4.5:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:freebsd:freebsd:4.6:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:freebsd:freebsd:4.7:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:linux:linux_kernel:2.4.1:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:linux:linux_kernel:2.4.2:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:linux:linux_kernel:2.4.3:*:*:*:*:*:*:*,CVE-2003-0001
0,cpe:2.3:o:linux:linux_kernel:2.4.4:*:*:*:*:*:*:*,CVE-2003-0001


In [ ]:
data=pd.read_json(file)
cve_items=data.CVE_Items.apply(pd.Series)
cve=cve_items.cve.apply(pd.Series)
config=cve_items.configurations.apply(pd.Series)
col=['part','vendor','product','version','update','edition','Language','Software_edition','Target_software','Target_hardware','Other']
import os
count=True
for i in range(len(config)):
    try:
        print(i)
        cpe=[]   
        vul=[]
        text=str(config['nodes'][i]).split(',')
        for row in text:
            if 'cpe23Uri' in row:
                cpe.append(row.strip().replace('}','').replace(']','').replace('\'','').replace('\'',''))
            if 'vulnerable' in row:
                vul.append(row.split(':')[-1].strip())
        if len(cpe)>0:
            df=pd.DataFrame(cpe)[0].str.split(':',n=50,expand=True).iloc[:,3:]
            df.columns=col
            df['Vulnerable']=vul
            df['CVE-ID']=metadata['ID'][i]
            #print(df)
        if count:
            a=df
            count=False
        else:
            
            a=a.append(df)
            #df.to_csv(os.getcwd()+os.sep+'NVD'+os.sep+'config'+os.sep+metadata['ID'][i]+'.csv')
    except:
        pass
